### Homework 03
#### 1. Building a neural network that computes XOR

It is impossible to use single-layer neural nets to solve the XOR, which is a reason for the disillusionment in the first generation of single-layer perceptrons. From our class lecture, single-layer neural nets require that the problem be linearly separable, such as AND or OR.
However, mapping out XOR, it is linearly inseparable. This makes it impossible to compute using a single-layer network which can only compute the output by multiplying some weights of the input. Instead, we need at least two layers, with a hidden layer that models the nonlinear behavior of XOR. Given that XOR is NAND ^ OR, we can make a two-layered neural network.
We can build the first layer taking the input, and at having two neurons that independently solve NAND and OR. The second layer contains a neuron that computes AND on the inputs from the first layer. This construction of a neural network will be able to produce the correct output. 

#### 2. Boston Housing data set
###### a. compute the dimensions of the data set:


In [18]:

import pandas as pd
import numpy as np
from keras.datasets import boston_housing

boston_housing = pd.read_csv("https://raw.githubusercontent.com/selva86/datasets/master/BostonHousing.csv")
boston_housing.head()
def print_structures(dataset):
    print(
        '   \tcount: {} \
            \n\tdimensions: {} \
            \n\tshape: {}\n\n'.format(
                len(dataset), 
                dataset.ndim, 
                dataset.shape      )
 )
print("Boston housing dataset dimensions: ")
print_structures(boston_housing)

Boston housing dataset dimensions: 
   	count: 506             
	dimensions: 2             
	shape: (506, 14)




###### b. creating training, validation, and testing sets

In [19]:
training = boston_housing[:300]
validation = boston_housing[300:400]
testing = boston_housing[400:]
print("training: ")
print_structures(training)
print("validation: ")
print_structures(validation)
print("testing: ")
print_structures(testing)

training: 
   	count: 300             
	dimensions: 2             
	shape: (300, 14)


validation: 
   	count: 100             
	dimensions: 2             
	shape: (100, 14)


testing: 
   	count: 106             
	dimensions: 2             
	shape: (106, 14)




###### c. introduce a new synthetic feature

In [21]:
print('Variables in the dataset:')
boston_housing.head()

Variables in the dataset:
      crim    zn  indus  chas    nox     rm   age     dis  rad  tax  ptratio  \
0  0.00632  18.0   2.31     0  0.538  6.575  65.2  4.0900    1  296     15.3   
1  0.02731   0.0   7.07     0  0.469  6.421  78.9  4.9671    2  242     17.8   
2  0.02729   0.0   7.07     0  0.469  7.185  61.1  4.9671    2  242     17.8   
3  0.03237   0.0   2.18     0  0.458  6.998  45.8  6.0622    3  222     18.7   
4  0.06905   0.0   2.18     0  0.458  7.147  54.2  6.0622    3  222     18.7   

        b  lstat  medv  
0  396.90   4.98  24.0  
1  396.90   9.14  21.6  
2  392.83   4.03  34.7  
3  394.63   2.94  33.4  
4  396.90   5.33  36.2  


Interested in the proportion between pupil-teacher ratio and per capita crime rate.
Could lower pupil-teacher ratio be a measure of a better educational system? Would this correlate with crime rates?
Because of these questions, I'm creating a synthetic feature that is the per capital crime divided by ptratio times a hundred.

In [27]:
boston_housing['education-crime-ratio'] = (boston_housing['crim']/boston_housing['ptratio'] * 100)
boston_housing.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat,medv,education-crime-ratio
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0,0.041307
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6,0.153427
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7,0.153315
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4,0.173102
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2,0.369251


In [26]:
boston_housing.describe()


,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat,medv,education-crime-ratio
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.613524,11.363636,11.136779,0.069170,0.554695,6.284634,68.574901,3.795043,9.549407,408.237154,18.455534,356.674032,12.653063,22.532806,18.198424
std,8.601545,23.322453,6.860353,0.253994,0.115878,0.702617,28.148861,2.105710,8.707259,168.537116,2.164946,91.294864,7.141062,9.197104,42.530956
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,0.320000,1.730000,5.000000,0.041307
25%,0.082045,0.000000,5.190000,0.000000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000,375.377500,6.950000,17.025000,0.456047
50%,0.256510,0.000000,9.690000,0.000000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000,391.440000,11.360000,21.200000,1.393775
75%,3.677082,12.500000,18.100000,0.000000,0.624000,6.623500,94.075000,5.188425,24.000000,666.000000,20.200000,396.225000,16.955000,25.000000,18.854352
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,37.970000,50.000000,440.476238
